In [ ]:
import numpy as np
import pandas as pd



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

# Step 1: Load CSV dataset
dataset_path = '/content/updated_hindi.csv'
df = pd.read_csv(dataset_path)



In [ ]:
df.dropna(inplace=True)

In [ ]:
class_names=['1','0']

In [ ]:
model_name = 'bert-base-multilingual-cased'

In [ ]:
# Load the Config object, with an output configured for classification.
config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name,
                                    num_labels=len(class_names))

print('Config type:', str(type(config)), '\n')


Config type: <class 'transformers.models.bert.configuration_bert.BertConfig'> 



In [ ]:
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoModel , AutoModelForSequenceClassification, AutoConfig , AutoTokenizer , AdamW
import torch
from tqdm import tqdm


# Load the BERT tokenizer.
print(f'Loading {model_name} tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)


class SarcasmDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'text': text,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Convert labels to numerical format
label_mapping = {label: idx for idx, label in enumerate(df['result'].unique())}
df['result'] = df['result'].map(label_mapping)

# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = SarcasmDataset(train_df['text'].tolist(), train_df['result'].tolist(), tokenizer, max_len=128)
val_dataset = SarcasmDataset(val_df['text'].tolist(), val_df['result'].tolist(), tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Step 4: Fine-tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name,config = config)
model.to(device)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # This is the value Michael used.
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_true_labels = []
    train_pred_labels = []

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        texts = batch['text']

        # for text, label, input_id, attn_mask in zip(texts, labels, input_ids, attention_mask):
        #   print(f"Text: {text}")
        #   print(f"Label: {label}")
          # print(f"Input IDs: {input_id}")
          # print(f"Attention Mask: {attn_mask}")

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # Record true labels and predicted labels for accuracy calculation
        train_true_labels.extend(labels.cpu().numpy())
        train_pred_labels.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())

        total_loss += loss.item()

    # Calculate accuracy for the epoch
    epoch_accuracy = accuracy_score(train_true_labels, train_pred_labels)

    # Print average loss and accuracy for the epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Training Accuracy: {epoch_accuracy * 100:.2f}%")

# Step 5: Evaluation
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


Loading bert-base-multilingual-cased tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/2: 100%|██████████| 1600/1600 [05:46<00:00,  4.61it/s]


Epoch 1/2, Average Training Loss: 0.1129, Training Accuracy: 95.98%


Epoch 2/2: 100%|██████████| 1600/1600 [05:44<00:00,  4.64it/s]


Epoch 2/2, Average Training Loss: 0.0465, Training Accuracy: 98.56%


Evaluating: 100%|██████████| 400/400 [00:24<00:00, 16.46it/s]

Validation Accuracy: 98.12%
